# Извлечение данных из коллекции новостных текстов

Данные расположены  [в файле data/news.tar.gz](data/news.tar.gz). С некоторых новостных сайтов был загружен архив новостей а  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`
    

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

        sport <tab> Сборная Канады по хоккею разгромила чехов <tab> Сборная Канады по хоккею крупно об...

С помощью [Yargy](https://github.com/natasha/yargy) или [Томита-парсера](https://github.com/yandex/tomita-parser) извлеките данные, которые можно описать структурой вида:


```python
@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]
```

**NOTE**: Если будет проблемы с `FileNotFoundError` и отображением деревьев, то мне помогло вот [это](https://giters.com/natasha/yargy/issues/82), а именно установка [версии 9.0.0](https://graphviz.org/download/) 

- Эстерхази родился в венгерской столице в 1950 году
- Иосиф Кобзон родился в городе Часов Яр Донецкой области
- Гвасалия родился в Грузии
- Трэмиел родился в 1928 году в Польше
- Салман Рушди родился в Мумбаи
- Он родился в Польше 11 сентября 1865 года
- Ребенок родился в Калифорнии 25 декабря
- принц Джордж, родился в 2013 году
- Мейстон родился 1935 году
- Дмитрий Чернявский родился в Артемовске 5 марта 1992 года
- Эли Визель родился в 1928 году в еврейской семье на севере Румынии.
- Миллиардер родился в 1938 году в Лондоне
- Юрий Яковлев родился в Москве в 1928 году
- Смирнов  18 декабря 1937 года в городе Калининграде (ныне Королев) Московской области.
- Патрик родился в Бронксе в 1990 году,
- Николай Караченцов родился 27 октября 1944 года в Москве в семье
- Орест I (он же Орест Федорович Карелин-Романишин-Русин) родился в 1950 году во Львове.
- Головко родился 24 ноября 1932 года в городе Бар (Винницкая область Украины)
- Том Роу (полное имя Томас Джон Роу - Thomas John Rowe) родился в 1956 году в Массачусетсе, США.
- Игорь Доценко родился в 1953 году в Хмельницкой области Украины.
- Энгельбарт родился в 1925 году в США
- Ежата Джордж, Александр и Луи родились в зоопарке 22 июля 2013 года.

In [6]:
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, is_capitalized, caseless, normalized, dictionary,  gte, lte
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.relations import gnc_relation


In [8]:


TITULS = morph_pipeline([
    "I",
    "II",
    "III",
    "IV",
    "V",
    "VI",
    "VII",
    "VIII",
    "IX",
    "X"
])

SURNNAME = or_(
    rule(gram("Surn")),
    rule(
        and_(
            is_capitalized(),
            gram("NOUN")
    )),
    rule(
        and_(
        is_capitalized(),
        gram("ADJF")
    )),
    rule(
        and_(
        is_capitalized(),
        gram("UNKN")
    )),
    rule(
        and_(
        is_capitalized(),
        gram("ADVB")
    )),
    rule(
        and_(
        is_capitalized(),
        gram("PRTF")
    )),
    TITULS
)

JUST_NAME = or_(
        rule(gram("Name")),
        rule(
           gram("Name"), "-", gram("Name")
        )     
)

NAME = or_(
    rule(
        JUST_NAME,
        SURNNAME.optional()
    ),
    rule(
        SURNNAME,
        JUST_NAME.optional()
    )
)


BIRTH_VERB = morph_pipeline([
    "родился",
    "появился на свет"
])

BIRTH_PLACE_ABBR = rule(
    gram("PREP").optional(),
    gram("Abbr")
)

BIRTH_PLACE_TAG = morph_pipeline([
    "штат",
    "город",
    "столица",
    "деревня",
    "окраина",
    "север",
    "юг",
    "восток",
    "запад",
    "край",
    "область",
    "республика",
    "область",
    "страна"
])



BIRTH_PLACE_FULL = rule(
    BIRTH_PLACE_TAG.optional(),
    or_(
        and_(
            is_capitalized(),
            gram("ADJF")
        ),
        and_(
            is_capitalized(),
            gram("NOUN")
        )
    ).repeatable(),
    BIRTH_PLACE_TAG.optional()
)



BIRTH_PLACE = rule(
    gram("PREP"),
    or_(
        BIRTH_PLACE_FULL.repeatable(),
        BIRTH_PLACE_ABBR
    )
)

DAY = and_(
    gte(1),
    lte(31)
)

MONTHS = dictionary({
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
})


YEAR = and_(
    gte(1),
    lte(2024)
)

YEAR_WORDS = morph_pipeline([
    "г.",
    "год"
])

DATE = or_(
    rule(YEAR, YEAR_WORDS.optional()),
    rule(
        DAY,
        MONTHS,
        YEAR,
        YEAR_WORDS.optional()
    ),
    rule(
        DAY,
        MONTHS
    )
)


BIRTH_DATE = rule(
    gram("PREP").optional(),
    DATE
)


S = rule(
    NAME,
    BIRTH_VERB,
    or_(
        rule(
            BIRTH_PLACE.repeatable(),
            BIRTH_DATE
        ),
        rule(
            BIRTH_DATE,
            BIRTH_PLACE.repeatable()
        )
    )
)


text = "Луи родился в зоопарке 22 июля 2013 года."   
text = text.replace(",", "")
import re
text = re.sub(r'\(.*?\)', '', text)
# print(text)


parser = Parser(S)
for match in parser.findall(text):
    print([_.value for _ in match.tokens])
    display(match.tree.as_dot)
    

In [9]:
import gzip

from dataclasses import dataclass
from typing import Iterator, Optional

@dataclass
class Text:
    label: str
    title: str
    text: str

@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("data/news.txt.gz"))

In [11]:
from tqdm import tqdm 

entries = []

def extract_entries(text: list[str]) -> list[Entry]:
    for text in tqdm(text, disable=True):
        text = text.text.replace(",", "")
        import re
        text = re.sub(r'\(.*?\)', '', text)
        try:
            for match in parser.findall(text):
                print([_.value for _ in match.tokens])
                entry = match.fact
                print(entry)
                entries.append({
                    'name': entry.person,
                    'birth_date': entry.birth_date,
                    'birth_place': entry.birth_place,
                })
        except:
            # print("Empty")
            pass
    return entries

data = extract_entries(texts)


['Бетси', 'Палмер', 'родилась', 'в', '1926', 'году', 'в', 'США']
Empty
['Трэмиел', 'родился', 'в', '1928', 'году', 'в', 'Польше']
Empty
['Ребенок', 'родился', 'в', 'Калифорнии', '25', 'декабря']
Empty
['Дмитрий', 'Чернявский', 'родился', 'в', 'Артемовске', '5', 'марта', '1992', 'года']
Empty
['Миллиардер', 'родился', 'в', '1938', 'году', 'в', 'Лондоне']
Empty
['ЯковлевЮрий', 'Яковлев', 'родился', 'в', 'Москве', 'в', '1928', 'году']
Empty
['Смирнов', 'родился', '18', 'декабря', '1937', 'года', 'в', 'городе', 'Калининграде', 'Московской', 'области']
Empty
['Патрик', 'родился', 'в', 'Бронксе', 'в', '1990', 'году']
Empty
['Николай', 'Караченцов', 'родился', '27', 'октября', '1944', 'года', 'в', 'Москве']
Empty
['Девочке', 'родившейся', 'с', 'ВИЧ', 'в', 'штате', 'Миссисипи', 'в', '2010', 'году']
Empty
['Орест', 'I', 'родился', 'в', '1950', 'году', 'во', 'Львове']
Empty
['Головко', 'родился', '24', 'ноября', '1932', 'года', 'в', 'городе', 'Бар']
Empty
['Живописец', 'родился', '25', 'мая', '1

**Вопросики:**

'Монро', 'родился', 'в', 'Ленинграде', 'в', '1969', 'году']

Девочке, родившейся с ВИЧ в штате Миссисипи в 2010 году